In [1]:
import pandas as pd
import time
import geopy
import datetime
from geopy.geocoders import Nominatim
df = pd.read_csv( "../data/AS1908-2016.csv")
def showcols(dataframe,lazyprinting = False):
    '''
    :param dataframe:
    :return nothing, but print out the dataframe column and corresponding indexes instead.
    The dataframe is not iterable through external package imports,
    so it must be defined within the jupyter kernel.
    '''
    if lazyprinting:
        print(dataframe.columns)
    for i in enumerate(dataframe.columns):
        print (i)

In [2]:
df
df[['Route']].describe()
a = enumerate(df.columns)
for i in a:
    print(i)

(0, 'Date')
(1, 'Time')
(2, 'Location')
(3, 'Operator')
(4, 'Flight #')
(5, 'Route')
(6, 'AC Type')
(7, 'Registration')
(8, 'cn/ln')
(9, 'Aboard')
(10, 'Aboard Passangers')
(11, 'Aboard Crew')
(12, 'Fatalities')
(13, 'Fatalities Passangers')
(14, 'Fatalities Crew')
(15, 'Ground')
(16, 'Summary')


In [3]:
da = df.drop(df.columns[[9,12,15]],axis = 1, inplace = False)
da
a = enumerate(da.columns)
for i in a:
    print(i)
da[['Aboard Passangers',
    'Fatalities Passangers',
    'Aboard Crew',
    'Fatalities Crew']].fillna(0,inplace = True)
da['Total Aborad'] = da['Aboard Passangers']+da['Aboard Crew']
da['Total Fatalities'] = da['Fatalities Crew'] + da['Fatalities Passangers']
da['Fatality Rate'] = da['Total Fatalities']/da['Total Aborad']

(0, 'Date')
(1, 'Time')
(2, 'Location')
(3, 'Operator')
(4, 'Flight #')
(5, 'Route')
(6, 'AC Type')
(7, 'Registration')
(8, 'cn/ln')
(9, 'Aboard Passangers')
(10, 'Aboard Crew')
(11, 'Fatalities Passangers')
(12, 'Fatalities Crew')
(13, 'Summary')


/anaconda3/envs/DataSci/lib/python3.6/site-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [4]:
da[['Total Aborad','Total Fatalities','Fatality Rate']]
da.drop(da[da['Fatality Rate'].isnull()].index,inplace = True)
da.drop(da[da['Fatality Rate']>1].index,inplace = True)
da.describe()
da = da.reset_index()
da.to_csv(path_or_buf='../data/cleanedsmall.csv',index = False)

In [8]:
da['Route'] = [str(i).lstrip('\t').strip() for i in da['Route']]
da['Raw_Time'] = da['Date'].apply(lambda x:time.strptime(x,"%m/%d/%Y"))
print(da.head(1))
da.info

   index        Date   Time             Location              Operator  \
0      0  09/17/1908  17:18  Fort Myer, Virginia  Military - U.S. Army   

  Flight #          Route           AC Type Registration cn/ln  \
0      NaN  Demonstration  Wright Flyer III          NaN     1   

   Aboard Passangers  Aboard Crew  Fatalities Passangers  Fatalities Crew  \
0                1.0          1.0                    1.0              0.0   

                                             Summary  Total Aborad  \
0  During a demonstration flight, a U.S. Army fly...           2.0   

   Total Fatalities  Fatality Rate                            Raw_Time  
0               1.0            0.5  (1908, 9, 17, 0, 0, 0, 3, 261, -1)  


<bound method DataFrame.info of       index        Date   Time                            Location  \
0         0  09/17/1908  17:18                 Fort Myer, Virginia   
1         1  09/07/1909    NaN             Juvisy-sur-Orge, France   
2         2  07/12/1912  06:30           Atlantic City, New Jersey   
3         3  08/06/1913    NaN  Victoria, British Columbia, Canada   
4        10  10/01/1916  23:45                Potters Bar, England   
...     ...         ...    ...                                 ...   
4704   4962  04/16/2019  11:00                 Puerto Montt, Chile   
4705   4963  05/05/2019  18:30               Near Monclava, Mexico   
4706   4964  05/05/2019  18:30                      Moscow, Russia   
4707   4965  06/03/2019  13:00                    Near Lipo, India   
4708   4966  07/30/2019  02:00                   Rawalpindi, India   

                                     Operator Flight #  \
0                        Military - U.S. Army      NaN   
1          

In [ ]:
geolocator = Nominatim(user_agent='capstoneproj')
from geopy.extra import rate_limiter
geocode=rate_limiter.RateLimiter(geolocator.geocode,min_delay_seconds= 1)
da['Coordinate'] = da['Location'].apply(geocode).apply(lambda location: (location.latitude, location.longitude))
print('Done')

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Near Bangkok, Thailand',), **{}).
Traceback (most recent call last):
  File "/home/ptuuu/anaconda3/envs/DS/lib/python3.6/site-packages/geopy/geocoders/base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/home/ptuuu/anaconda3/envs/DS/lib/python3.6/urllib/request.py", line 526, in open
    response = self._open(req, data)
  File "/home/ptuuu/anaconda3/envs/DS/lib/python3.6/urllib/request.py", line 544, in _open
    '_open', req)
  File "/home/ptuuu/anaconda3/envs/DS/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)
  File "/home/ptuuu/anaconda3/envs/DS/lib/python3.6/urllib/request.py", line 1361, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/home/ptuuu/anaconda3/envs/DS/lib/python3.6/urllib/request.py", line 1321, in do_open
    r = h.getresponse()
  File "/home/ptuuu/anaconda3/envs/DS/lib/python3.6/ht

In [ ]:
print('Hi')


georec = []
for i in range(0,len(da['Location'])):
    geolocator = Nominatim(user_agent='capstoneproj')
    location = geolocator.geocode(da['Location'][i])
    georec.append(location)
    time.sleep(1)
print('Done')

